In [1]:
import io

import pyterrier as pt
from experiment_utils.experiments_helper import get_timeit_dependencies_name, getOptimalAlpha, latency_per_query

if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
from encoders.bge_base_en import BgeQueryEncoder

package = "BAAI/"
model_name = "bge-base-en-v1.5"
q_encoder = BgeQueryEncoder(package + model_name)
model_directory = 'bge'
path_to_root = "../../"
pipeline_name = "BM25 >> bge-base-en-v1.5"

In [3]:
prefix = "irds:beir/"
test_suffix = "/test"

In [4]:
def get_timeit_dep(dataset_name):
    return get_timeit_dependencies_name(prefix + dataset_name, prefix + dataset_name + test_suffix,
                                        q_encoder,
                                        model_name,
                                        path_to_root, model_directory,
                                        alpha=getOptimalAlpha(prefix + dataset_name, pipeline_name, model_directory))

In [5]:
from contextlib import redirect_stdout


def measure_latency(dataset_name):
    results_lexical_retriever, semantic_reranker = get_timeit_dep(dataset_name)
    f = io.StringIO()
    with redirect_stdout(f):
        %timeit semantic_reranker(results_lexical_retriever)

    timeit_output = f.getvalue()
    result = latency_per_query(timeit_output, prefix + dataset_name, test_suffix, pipeline_name, model_directory)
    print(result)

In [6]:
dataset_list = ["nfcorpus", "fiqa", "scifact"]

In [7]:
import traceback


def run_latency_datasets():
    for dataset_name in dataset_list:
        try:
            measure_latency(dataset_name)
        except Exception as e:
            traceback.print_exc()

In [ ]:
run_latency_datasets()

100%|██████████| 3633/3633 [00:00<00:00, 1568815.65it/s]


Latency per query: 19.4427 ms. Experiment details: 6.28 s +- 91.3 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 57638/57638 [00:00<00:00, 1570466.26it/s]


In [10]:
model_name = "bge-small-en-v1.5"
pipeline_name = "BM25 >> " + model_name
q_encoder = BgeQueryEncoder(package + model_name)

run_latency_datasets()

100%|██████████| 3633/3633 [00:00<00:00, 1598270.03it/s]


Latency per query: 8.7307 ms. Experiment details: 2.82 s +- 133 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



In [ ]:
from encoders.gte_base_en_encoder import GTEBaseEncoder

package = "Alibaba-NLP/"
model_name = "gte-base-en-v1.5"
q_encoder = GTEBaseEncoder(package + model_name)
model_directory = 'gte_base_en_v1_5'
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

In [ ]:
from encoders.snowflake_arctic_embed_m import SnowFlakeQueryEncoder

package = "Snowflake/"
model_name = "snowflake-arctic-embed-m"
q_encoder = SnowFlakeQueryEncoder(package + model_name)
model_directory = 'snowflake'
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

In [ ]:
model_name = "snowflake-arctic-embed-xs"
q_encoder = SnowFlakeQueryEncoder(package + model_name)

for dataset_name in dataset_list:
    measure_latency(dataset_name)

In [ ]:
from fast_forward.encoder import TCTColBERTQueryEncoder

package = "castorini/"
model_name = "tct_colbert-msmarco"
q_encoder = TCTColBERTQueryEncoder(package + model_name)
model_directory = 'tct_colbert'
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()